In [13]:
import numpy as np
from pymatgen.io.cif import CifParser
from pymatgen.core import Structure
from pymatgen.core.lattice import Lattice

In [14]:
import sys
sys.path.append('../..')
from ai4mat.data.data import (
    StorageResolver)

In [15]:
from ase.visualize import view
from pymatgen.io.ase import AseAtomsAdaptor

In [16]:
mp_cell = CifParser(StorageResolver()["materials"] / 
    "P_mp-1014013_computed.cif").get_structures(primitive=False)[0]

In [17]:
view(AseAtomsAdaptor.get_atoms(mp_cell), viewer='ngl')

In [18]:
mp_cell

Structure Summary
Lattice
    abc : 3.299918 4.600926 35.235589
 angles : 90.0 90.0 90.0
 volume : 534.9706203931867
      A : 3.299918 0.0 2.0206170080743677e-16
      B : -2.817254649506918e-16 4.600926 2.817254649506918e-16
      C : 0.0 0.0 35.235589
    pbc : True True True
PeriodicSite: P (-0.0000, 2.7079, 34.1825) [0.0000, 0.5886, 0.9701]
PeriodicSite: P (-0.0000, 1.8930, 1.0531) [0.0000, 0.4114, 0.0299]
PeriodicSite: P (1.6500, 0.4074, 1.0531) [0.5000, 0.0886, 0.0299]
PeriodicSite: P (1.6500, 4.1935, 34.1825) [0.5000, 0.9114, 0.9701]

https://www.researchgate.net/figure/a-Atomic-structure-of-multi-layer-black-phosphorus-and-b-monolayer-phosphorene_fig1_280631658

![Black phosphorus monolayer](https://www.researchgate.net/profile/Xi-Lin-23/publication/280631658/figure/fig1/AS:379933193981953@1467594540417/a-Atomic-structure-of-multi-layer-black-phosphorus-and-b-monolayer-phosphorene.png)

In [25]:
# In principle, we could use the above cell and let pbc take care of the armchair structures
# But we'll make an explicit layer

In [28]:
double_cell = mp_cell.copy()
double_cell.make_supercell([1, 1, 2])

In [29]:
view(AseAtomsAdaptor.get_atoms(double_cell), viewer='ngl')

In [30]:
double_supercell = double_cell.copy()
double_supercell.make_supercell([6, 6, 1])

In [33]:
view(AseAtomsAdaptor.get_atoms(double_supercell), viewer='ngl')

In [35]:
selection = (double_cell.frac_coords[:, 2] < 0.7) & (double_cell.frac_coords[:, 2] > 0.3)
assert np.count_nonzero(selection)*2 == len(double_cell)

In [44]:
double_cell

Structure Summary
Lattice
    abc : 3.299918 4.600926 70.471178
 angles : 90.0 90.0 90.0
 volume : 1069.9412407863733
      A : 3.299918 0.0 2.0206170080743677e-16
      B : -2.817254649506918e-16 4.600926 2.817254649506918e-16
      C : 0.0 0.0 70.471178
    pbc : True True True
PeriodicSite: P (-0.0000, 2.7079, 34.1825) [0.0000, 0.5886, 0.4851]
PeriodicSite: P (-0.0000, 2.7079, 69.4181) [0.0000, 0.5886, 0.9851]
PeriodicSite: P (-0.0000, 1.8930, 1.0531) [0.0000, 0.4114, 0.0149]
PeriodicSite: P (-0.0000, 1.8930, 36.2887) [0.0000, 0.4114, 0.5149]
PeriodicSite: P (1.6500, 0.4074, 1.0531) [0.5000, 0.0886, 0.0149]
PeriodicSite: P (1.6500, 0.4074, 36.2887) [0.5000, 0.0886, 0.5149]
PeriodicSite: P (1.6500, 4.1935, 34.1825) [0.5000, 0.9114, 0.4851]
PeriodicSite: P (1.6500, 4.1935, 69.4181) [0.5000, 0.9114, 0.9851]

In [48]:
selection

array([ True, False, False,  True, False,  True,  True, False])

In [51]:
monolayer = Structure.from_sites([double_cell[i] for i in np.where(selection)[0]])
monolayer

Structure Summary
Lattice
    abc : 3.299918 4.600926 70.471178
 angles : 90.0 90.0 90.0
 volume : 1069.9412407863733
      A : 3.299918 0.0 2.0206170080743677e-16
      B : -2.817254649506918e-16 4.600926 2.817254649506918e-16
      C : 0.0 0.0 70.471178
    pbc : True True True
PeriodicSite: P (-0.0000, 2.7079, 34.1825) [0.0000, 0.5886, 0.4851]
PeriodicSite: P (-0.0000, 1.8930, 36.2887) [0.0000, 0.4114, 0.5149]
PeriodicSite: P (1.6500, 0.4074, 36.2887) [0.5000, 0.0886, 0.5149]
PeriodicSite: P (1.6500, 4.1935, 34.1825) [0.5000, 0.9114, 0.4851]

In [52]:
monolayer.cart_coords

array([[-1.65811494e-16,  2.70790720e+00,  3.41824677e+01],
       [-1.15913970e-16,  1.89301880e+00,  3.62887103e+01],
       [ 1.64995900e+00,  4.07444204e-01,  3.62887103e+01],
       [ 1.64995900e+00,  4.19348180e+00,  3.41824677e+01]])

In [53]:
lattice_np = np.array(monolayer.lattice.matrix)
lattice_np[2, 2] = 20
tall_lattice = Lattice(lattice_np)

reference_cell = Structure(
    lattice=tall_lattice,
    species=monolayer.species,
    coords=monolayer.cart_coords - np.array([0, 0, 30]),
    coords_are_cartesian=True,
)

In [54]:
reference_cell

Structure Summary
Lattice
    abc : 3.299918 4.600926 20.0
 angles : 90.0 90.0 90.0
 volume : 303.65357048136
      A : 3.299918 0.0 2.0206170080743677e-16
      B : -2.817254649506918e-16 4.600926 2.817254649506918e-16
      C : 0.0 0.0 20.0
    pbc : True True True
PeriodicSite: P (-0.0000, 2.7079, 4.1825) [-0.0000, 0.5886, 0.2091]
PeriodicSite: P (-0.0000, 1.8930, 6.2887) [-0.0000, 0.4114, 0.3144]
PeriodicSite: P (1.6500, 0.4074, 6.2887) [0.5000, 0.0886, 0.3144]
PeriodicSite: P (1.6500, 4.1935, 4.1825) [0.5000, 0.9114, 0.2091]

In [55]:
from pymatgen.io.cif import CifWriter
CifWriter(reference_cell).write_file(StorageResolver()["materials"] /"P.cif")

In [56]:
supercell = reference_cell.copy()
supercell.make_supercell((6, 6, 1))

In [57]:
view(AseAtomsAdaptor.get_atoms(supercell), viewer='ngl')